In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Flatten, GlobalAveragePooling2D, Concatenate, Reshape
from tensorflow.keras.models import Sequential

In [2]:
import csv
images = []
image_path = 'HARRISON_full/HARRISON/data_list.txt'
with open(image_path, 'r') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)
    for row in reader:
        image_path = row[0]
        images.append(image_path)
prefix = "D:/pkapu/Documents/Summer Internship/Listed/HARRISON_full/HARRISON/"
images = [prefix + item for item in images]
images = images[:2000]

In [3]:
print(images[0])

D:/pkapu/Documents/Summer Internship/Listed/HARRISON_full/HARRISON/instagram_dataset/sea/image_1284.jpg


In [4]:
hashtags = []
hash_path = 'HARRISON_full/HARRISON/tag_list.txt'
with open(hash_path, 'r') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)
    for row in reader:
        hash_path = row[0].split()
        hashtags.append(hash_path)
hashtags = hashtags[:2000]

In [5]:
import cv2
import numpy as np

def preprocess_image(image_data):
    processed_images = []
    if isinstance(image_data, list):
        for image_path in image_data:
            img = cv2.imread(image_path)
            if img is not None:
                img = cv2.resize(img, (224, 224))
                img = img / 255.0
                processed_images.append(img)
            else:
                print(f"Error loading image: {image_path}")
    else:
        img = cv2.imread(image_data)
        if img is not None:
            img = cv2.resize(img, (224, 224))
            img = img / 255.0
            processed_images.append(img)
        else:
            print(f"Error loading image: {image_data}")
    return np.array(processed_images)

In [6]:
def preprocess_hashtags(hashtags):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(hashtags)
    sequences = tokenizer.texts_to_sequences(hashtags)
    word_index = tokenizer.word_index
    return sequences, word_index

In [7]:
def build_model(vocab_size, max_sequence_length):
    # Image encoder
    image_input = Input(shape=(224, 224, 3))  # Adjust input shape as per your model's input size
    image_model = VGG16(weights='imagenet', include_top=False)(image_input)
    image_features = Flatten()(image_model)

    # Hashtag generator
    hashtag_input = Input(shape=(max_sequence_length,))
    embedding_layer = Embedding(vocab_size, 256, input_length=max_sequence_length)(hashtag_input)
    lstm = LSTM(256)(embedding_layer)
    output = Dense(vocab_size, activation='softmax')(lstm)

    model = Model(inputs=[image_input, hashtag_input], outputs=output)
    return model

In [8]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def prepare_data(images, hashtags, word_index, max_sequence_length):
    start_token = '<start>'
    if start_token not in word_index:
        word_index[start_token] = len(word_index) + 1
    X_image = []  # List to store image data
    X_hashtag = []  # List to store hashtag data
    y = []  # List to store target labels

    for image, hashtag_list in zip(images, hashtags):
        # Process image data (e.g., using preprocess_image() function)
        image_data = preprocess_image(image)
        X_image.append(image_data)

        # Process hashtag data (convert words to indices)
        hashtag_indices = [word_index.get(word, 0) for word in hashtag_list]  # Use 0 for unknown words
        X_hashtag.append(hashtag_indices)

        # Prepare target labels (one-hot encoding or other suitable encoding)
        target = [0] * vocab_size  # Initialize with all zeros
        for index in hashtag_indices:
            target[index] = 1
        y.append(target)

    # Convert lists to numpy arrays
    X_image = np.array(X_image)
    X_hashtag = pad_sequences(X_hashtag, maxlen=max_sequence_length)

    y = np.array(y)

    return X_image, X_hashtag, y

In [9]:
def evaluate_model(model, X_image, X_hashtag, y):
    loss, accuracy = model.evaluate([X_image, X_hashtag], y)
    print("Loss: {:.4f}".format(loss))
    print("Accuracy: {:.2f}%".format(accuracy * 100))

In [10]:
def predict_hashtags(model, image_path, word_index, max_sequence_length, num_hashtags=10):
    image = preprocess_image(image_path)
    image = np.expand_dims(image, axis=0)
    
    sequence_input = np.zeros((1, max_sequence_length))
    sequence_input[0, 0] = word_index['<start>']  # Start token

    for i in range(1, max_sequence_length):
        predictions = model.predict([image, sequence_input])
        predicted_word_index = np.argmax(predictions[0, i-1])
        sequence_input[0, i] = predicted_word_index

    predicted_hashtags = []
    for index in sequence_input[0]:
        if index == 0:  # End token
            break
        word = list(word_index.keys())[list(word_index.values()).index(index)]
        predicted_hashtags.append(word)

    return predicted_hashtags[:num_hashtags]

In [13]:
sequences, word_index = preprocess_hashtags(hashtags)
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(hashtags)

In [14]:
sequences, word_index = preprocess_hashtags(hashtags)
X_image = preprocess_image(images)
max_sequence_length = 20  # Maximum sequence length for hashtags
vocab_size = len(word_index) + 1
X_image, X_hashtag, y = prepare_data(images, hashtags, word_index, max_sequence_length)
X_image = np.squeeze(X_image, axis=1)
num_classes = 10  # Number of classes
y_encoded = tf.keras.utils.to_categorical(y, num_classes=num_classes)

# Define your model architecture
image_input = Input(shape=(224, 224, 3))
hashtag_input = Input(shape=(max_sequence_length,))
flattened_hashtags = Embedding(input_dim=len(word_index) + 1, output_dim=100, input_length=max_sequence_length)(hashtag_input)
flattened_hashtags = Reshape((max_sequence_length, 100, 1))(flattened_hashtags)
flattened_hashtags = GlobalAveragePooling2D()(flattened_hashtags)
flattened_image = GlobalAveragePooling2D()(image_input)
concatenated_inputs = Concatenate(axis=-1)([flattened_image, flattened_hashtags])
dense_layer = Dense(256, activation='relu')(concatenated_inputs)
output_layer = Dense(num_classes, activation='softmax')(dense_layer)

model = Model(inputs=[image_input, hashtag_input], outputs=output_layer)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit([X_image, X_hashtag], y_encoded[:, 0, :], epochs=10, batch_size=32)
# Perform inference
test_image = images[2001:2500]  # Test image data
test_hashtag = hashtags[2001:2500]  # Test hashtag sequence
# Preprocess the test image
test_image_data = preprocess_image(test_image)  # Assuming test_image_path is the path to your test image

# Ensure test_image_data has the correct shape
test_image_data = test_image_data.reshape((-1, 224, 224, 3))

# Generate padded test hashtag sequence
test_hashtag_sequence = ["<start> " + hashtag + " <end>" for hashtag in test_hashtag]
test_hashtag_sequence = tokenizer.texts_to_sequences(test_hashtag_sequence)
padded_test_hashtag = pad_sequences(test_hashtag_sequence, maxlen=max_sequence_length, padding='post')

# Repeat the test image data to match the number of samples in padded_test_hashtag
test_image_data = np.repeat(test_image_data, padded_test_hashtag.shape[0], axis=0)

# Make predictions using the model
predictions = model.predict([test_image_data, padded_test_hashtag])



# Retrieve top 10 predicted hashtags
top_10_hashtags = [hashtag for _, hashtag in sorted(zip(predictions[0], word_index.keys()), reverse=True)[:10]]
print(top_10_hashtags)

Epoch 1/10
63/63 [==============================] - 2s 8ms/step - loss: 1.1549 - accuracy: 0.9370
Epoch 2/10
63/63 [==============================] - 1s 8ms/step - loss: 0.0624 - accuracy: 1.0000
Epoch 3/10
63/63 [==============================] - 1s 9ms/step - loss: 0.0090 - accuracy: 1.0000
Epoch 4/10
63/63 [==============================] - 1s 9ms/step - loss: 0.0026 - accuracy: 1.0000
Epoch 5/10
63/63 [==============================] - 1s 9ms/step - loss: 9.8208e-04 - accuracy: 1.0000
Epoch 6/10
63/63 [==============================] - 1s 8ms/step - loss: 4.9760e-04 - accuracy: 1.0000
Epoch 7/10
63/63 [==============================] - 1s 8ms/step - loss: 2.9827e-04 - accuracy: 1.0000
Epoch 8/10
63/63 [==============================] - 1s 8ms/step - loss: 1.9743e-04 - accuracy: 1.0000
Epoch 9/10
63/63 [==============================] - 1s 8ms/step - loss: 1.3840e-04 - accuracy: 1.0000
Epoch 10/10
63/63 [==============================] - 0s 8ms/step - loss: 1.0076e-04 - accuracy: 1.

ValueError: Unexpected result of `predict_function` (Empty batch_outputs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.

In [18]:
print(images[0])

D:/pkapu/Documents/Summer Internship/Listed/HARRISON_full/HARRISON/instagram_dataset/sea/image_1284.jpg


In [23]:
sequences, word_index = preprocess_hashtags(hashtags)

vocab_size = len(word_index) + 1  # Add 1 for the padding token
max_sequence_length = 10  # Adjust as per your dataset

# Step 4: Training
X_image, X_hashtag, y = prepare_data(images, hashtags, word_index, max_sequence_length)
model = build_model(vocab_size, max_sequence_length)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit([X_image, X_hashtag], y, batch_size=32, epochs=10, validation_split=0.2)


Epoch 1/10
13/13 [==============================] - 3s 85ms/step - loss: 29.6407 - accuracy: 0.9000 - val_loss: 30.5022 - val_accuracy: 0.9700
Epoch 2/10
13/13 [==============================] - 1s 42ms/step - loss: 36.7097 - accuracy: 0.9825 - val_loss: 38.4855 - val_accuracy: 0.9700
Epoch 3/10
13/13 [==============================] - 1s 41ms/step - loss: 37.0907 - accuracy: 0.9825 - val_loss: 39.3527 - val_accuracy: 0.9700
Epoch 4/10
13/13 [==============================] - 1s 43ms/step - loss: 39.0847 - accuracy: 0.9825 - val_loss: 44.1943 - val_accuracy: 0.9700
Epoch 5/10
13/13 [==============================] - 1s 42ms/step - loss: 43.2756 - accuracy: 0.9825 - val_loss: 48.8827 - val_accuracy: 0.9700
Epoch 6/10
13/13 [==============================] - 1s 44ms/step - loss: 46.9097 - accuracy: 0.9825 - val_loss: 52.7451 - val_accuracy: 0.9700
Epoch 7/10
13/13 [==============================] - 1s 45ms/step - loss: 50.1517 - accuracy: 0.9825 - val_loss: 56.9113 - val_accuracy: 0.9700

In [24]:
evaluate_model(model, X_image, X_hashtag, y)

16/16 [==============================] - 0s 15ms/step - loss: 64.5801 - accuracy: 0.9800
Loss: 64.5801
Accuracy: 98.00%


In [25]:
new_image_path = 'D:/pkapu/Documents/Summer Internship/Listed/HARRISON_full/HARRISON/instagram_dataset/sea/image_1315.jpg'  # Replace with your image path
predicted_hashtags = predict_hashtags(model, new_image_path, word_index, max_sequence_length)
print("Predicted Hashtags:", predicted_hashtags)

InvalidArgumentError: Graph execution error:

Detected at node 'model_1/embedding_1/embedding_lookup' defined at (most recent call last):
    File "C:\Users\pkapu\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\pkapu\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\pkapu\anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\pkapu\anaconda3\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "C:\Users\pkapu\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 711, in start
      self.io_loop.start()
    File "C:\Users\pkapu\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\pkapu\anaconda3\lib\asyncio\base_events.py", line 601, in run_forever
      self.